# 598 DLH - Team 30 Final Project Report
**Name(s):** Kudzai Bishi

**Email(s):** kbishi2@illinois.edu

**Project GitHub Repo:** https://github.com/598team30/DL4H_Team_30

**Project Google Drive:** https://drive.google.com/drive/folders/1IfqKghs9ztuYZfVYV_mHXAvdZ6q-J2JU (contains video and word embeddings)


-------------------------------

**Citation to the original paper:**
Bardak, B., & Tan, M. (2021). Improving clinical outcome predictions using convolution over medical entities with multimodal learning. Artificial intelligence in medicine, 117, 102112. https://doi.org/10.1016/j.artmed.2021.102112

**Original papers GitHub link:** https://github.com/tanlab/ConvolutionMedicalNer



# Introduction

### Background of the problem

Accurately predicting mortality and length of stay can help manage hospital resources, improve patient care, and save lives. After the release of the MIMIC-III (Medical Information Mart for Intensive Care) [1] EHR database, many studies utilized deep learning models on this database to predict different clinical outcomes. However, most studies mainly focused on using structured EHR data like diagnosis codes, vital signs, and lab results rather than unstructured clinical notes. This is because clinical notes are difficult to process due to being high dimensional and noisy. Although clinical notes present challenges, they are very detailed, containing various information about patients and their health status. This information could enhance vital healthcare predictions.

The paper addresses this need to improve clinical outcomes by tackling the following types of problems: multimodal mortality (in-hospital and in-ICU) and in-ICU length of stay (>3 and >7 days) prediction (including data processing, feature engineering, and data integration). The author’s main goal was to find effective ways to integrate time series and clinical notes.

### Paper explanation

The paper proposes a novel convolutional-based multimodal deep learning architecture that uses time series and medical entity features to improve mortality (in-hospital and in-ICU) and in-ICU length of stay (>3 and >7 days) predictions. Key features of the proposed approach include:
-	Extracting time-series variables using ICU vital signals and lab results from MIMIC-Extract [2], an open-source data extraction pipeline,
-	Extracting medical entities from MIMIC-III clinical notes using med7 [3], a pre-trained clinical Named Entity Recognition (NER) model,
-	Exploring different word embeddings on the medical entities, namely pre-trained Word2Vec, FastText, and a concatenation of both,
-	Utilizing a 3-layer 1D Convolutional Neural Network (CNN), with a max pooling layer to extract features from the medical entities and a single layer Gated Recurrent Unit (GRU) to extract features from the time series data,
-	Lastly, combining the time series and medical entity features and then feeding them into a fully connected layer to produce the predictions.


![Proposed Model Architecture](https://ars.els-cdn.com/content/image/1-s2.0-S0933365721001056-gr2_lrg.jpg)


> Fig. 1. Overview of Proposed multimodal architecture for predicting mortality and length of stay.  

The proposed method outperformed both a unimodal baseline model based on time series data alone and a multimodal baseline using averaged word embeddings (no CNN) on all prediction AUROC, AUPRC and F1 scores except for F1 score for length of stay > 7 days.

This paper is important to the problem as it presents a  way to improve clinical outcome prediction accuracy by addressing challenges in data processing, feature engineering, and data integration.



# Scope of Reproducibility

Hypotheses to be tested and the corresponding experiments to be run.

**Hypothesis:** A multimodal deep learning approach of combining embedded medical entity features from a CNN and time series features from a GRU, trained using MIMIC-III data, will outperform both a unimodal approach of solely using the time series GRU and a multimodal approach without convolution on predicting mortality and length-of-stay.

**Experiments:**
1. Implement the proposed model (training, evaluation and testing).
:
**Ablations:**
1. Model trained without convolution on medical entities (asses the effect of the convolution layer)
2. Model trained without time series and model trained without medical entities (assess the effect of multimodal approach)

**Additional Experiment:**
1. Hyperparameter Tuning.

# Set Path/Mount Notebook to Google Drive

This will mount to drive if using Google Colab.

Note: running this will produce a popup that requires manual selections/authorization to proceed.

In [1]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive') # this is apparently sensitive to double or single quotes
    PROJECT_PATH = "/content/gdrive/My Drive/Colab Notebooks/DL4H Team 30 Project/"
else:
    PROJECT_PATH = ""

REQUIREMENTS_FILE = f"{PROJECT_PATH}requirements.txt"

Mounted at /content/gdrive


# Methodology

This section consists of runnable code with necessary annotations to show the executed experiment for testing the hypotheses. Includes the main sections **Environment**, **Data** and **Model** and corresponding subsections.

## Environment

### Python version

Python Version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]

In [2]:
print("Python Version:", sys.version)

Python Version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


###  Install packages

In [3]:
pip install -r "{REQUIREMENTS_FILE}"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.4/607.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.1
    Uninstalling pydantic-2.7.1:
      Successfully uninstalled pydantic-2.7.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.3
    Uninstalling thinc-8.2.3:
      Successfully uninstalled t

###  Import packages

In [4]:
# import packages needed
import pandas as pd
import os
import numpy as np
import re
from statistics import mean
from gensim.models import Word2Vec, FastText
from mittens import GloVe
import collections
import nltk
import spacy
import time
import random
import gc

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D

from tensorflow.keras.optimizers.legacy import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import to_categorical
from tensorflow.keras.backend import clear_session
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.regularizers import L2
import tensorflow as tf

from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

from IPython.display import display, HTML

import warnings
warnings.filterwarnings('ignore')


In [5]:
# use only the first 24 hour of patient data after ICU admission
window_size = 24
# only consider patients with at least 30 hours of present data
min_present_data = 30
# set subsample data size
sample_size = 100 # number of patients. Can put None to use full data, this will require 3+ days and ~ 24GB RAM

# set seed
SEED = 10
np.random.seed(SEED)

def set_seed(seed):
    """Set seed"""
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(SEED)

# Define data path
DATA_PATH = f"{PROJECT_PATH}data"
EMBEDDING_PATH = f"{PROJECT_PATH}embeddings"
RESULTS_PATH = f"{PROJECT_PATH}results"

In [6]:
# start measuring runtime (not considering package installation)
_START_RUNTIME = time.time()

##  Data
This section includes raw data access instructions (MIMIC-Extract and MIMIC-III tables), descriptive statistics and data processing.

### Data access and download instructions
Can also find download instrcutions here: [GitHub README.md](https://github.com/598team30/DL4H_Team_30/blob/main/README.md#datasets ).

####  MIMIC-III
MIMIC-III ('Medical Information Mart for Intensive Care') [1] is a large, publically available database containing de-identified health-related data associated with over 45,000 patients who stayed in the Beth Israel Deaconess Medical Center ICU between 2001 and 2012. (see exact  statistics printed under section "MIMIC-III clinical notes")

**To obtain access to the dataset:**

1. Navigate to the MIMIC-III [1] PhysioNet page: https://physionet.org/content/mimiciii/1.4/
2. Navigate to the `Files` section at the bottom of the page.
3. Follow the instructions provided.
4. After getting access, repeat steps 1 and 2. All files should now be accessible for download.
5. Download `ADMISSIONS.csv.gz`, `ICUSTAYS.csv.gz` and `NOTEEVENTS.csv.gz`.
6. Unzip each file.

`ADMISSIONS.csv`, `ICUSTAYS.csv` and `NOTEEVENTS.csv` are the hospital admission data, ICU admission data and ICU clinical note data, respectively.

####  MIMIC-Extract
MIMIC-Extract [2] is an open-source data extraction and preprocessing pipeline that transforms raw MIMIC-III EHR data into datasets suitable for time-series prediction tasks. It includes patient ICU stays with the following criteria: the patient is at least 15 at the time of admission, the stay is the first known ICU admission for that patient, and the length of stay is between 12 hours and 10 days. The pipeline produces a cohort of 34,472 patients and 104 clinically aggregated time-series variables. (see printed statistics under section "MIMIC-Extract time series data")

**To obtain access to the dataset:**

Access and download the pre-processed output from Google Cloud Platform.
1. Get access to MIMIC-III as outlined above
2. Link your email account to your PhysioNet profile https://mimic.mit.edu/docs/gettingstarted/cloud/link/.
2. Request access to the cloud resource for MIMIC-III as outlined in this link: https://mimic.mit.edu/docs/gettingstarted/cloud/request/.
3. Click the access link to the Google Cloud Platform storage bucket sent via email.
4. Navigate to the GitHub page MIMIC-Extract [2] https://github.com/MLforHealth/MIMIC_Extract.
5. Navigate to the “Pre-processed Output” section and clicked the link provided for Google Cloud Platform (referred to as gcp).
6. Download the data `all_hourly_data.zip`.
7. Unzip each file

`all_hourly_data.csv` contains the processed time series data.

### Useful data definitions

Reference: https://mimic.mit.edu/docs/iii/

**Columns:**
- SUBJECT_ID: identifies a unique patient
- HADM_ID: identifies a unique admission to the hospital
- ICUSTAY_ID: identifies a unique admission to the ICU
- INTIME: time when the patient entered the ICU
- OUTTIME: time when the patient left the ICU
- CHARTTIME: time when measurements were documented/charted (can be a proxy for when the measurement was taken)


### MIMIC-Extract time series data

* Note this data took a while to load to colab (I did this very early on before the midterm)

In [ ]:
# dir and function to load raw data
raw_mimic_extract_data_dir = f"{DATA_PATH}/all_hourly_data.h5"

def load_raw_mimic_extract_data(raw_data_dir):
    # load vital labs (ICU level data)
    admissions = pd.read_hdf(raw_data_dir, 'vitals_labs')
    #print('vital labs done')
    # load patient level data
    patients = pd.read_hdf(raw_data_dir, 'patients')
    return admissions, patients

# calculate statistics
def calculate_stats(raw_admissions_data, raw_patients_data):
    # implement this function to calculate the statistics
    print(" Statistics of MIMIC-Extract raw data:")
    print("   - ", "Size/shape of dataset:", raw_admissions_data.shape)
    # below match statistics in paper
    print("   - ","Total # of patients:", len(raw_admissions_data.groupby('subject_id')))
    print("   - ","Total # of hospital admissions:", len(raw_admissions_data.index.get_level_values('hadm_id').unique()))
    print("   - ","Total # of ICU admissions:", len(raw_admissions_data.index.get_level_values('icustay_id').unique()))
    print("   - ","Total # clinically aggregated time-series variables:", len(raw_admissions_data.columns.levels[0].unique()))
    # ['mort_hosp', 'mort_icu', 'los_icu']
    print ("   - In-hospital mortality distribution:",
           round(sum(raw_patients_data.mort_hosp.values)*100 / len(raw_patients_data.mort_hosp.values),1),"%")
    print ("   - In-ICU mortality distribution:",
           round(sum(raw_patients_data.mort_icu.values)*100 / len(raw_patients_data.mort_icu.values),1),"%")
    print ("   - Length-of-stay > 3 distribution:",
           round(sum(raw_patients_data['los_icu'] > 3)*100 / len(raw_patients_data.los_icu.values),1),"%")
    print ("   - Length-of-stay > 7 distribution:",
           round(sum(raw_patients_data['los_icu'] > 7)*100 / len(raw_patients_data.los_icu.values),1),"%")
    print( "NOTE: the above 4 distributions do not all match the final cohort distribution.")

# process raw data
def simple_imputer(df):
    """
    Impute missing values (not mentioned in the paper, but it is part of the code GitHub repo which
    looks to be sourced from scripts on the MIMIC-Extract GitHub page).
    """
    id_columns = ['subject_id', 'hadm_id', 'icustay_id']
    idx = pd.IndexSlice
    df = df.copy()
    # drop the first 2 levels
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    # fill missing ICU means, first with forward fill, then group averages, then filling the remaining with 0
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(id_columns).mean()

    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(id_columns).fillna(
        method='ffill'
    ).groupby(id_columns).fillna(icustay_means).fillna(0)

    # create a mask column as an indicator for when the count is greater than 0
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    # ??
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum() #cumulative sum
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    # fill in missing time since measured with 100
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)

    # sort the dataframe by column names
    df_out.sort_index(axis=1, inplace=True)
    return df_out

def process_time_series_data(admissions, patients):
    """
    Take a sample of the datasets and process as described in paper:
        1. Subsample the data
        2. Only consider the patients with at least 30 h of present data
        3. Create boolean lables for length of stay >3 and >7 targets
        4. Use the first 24 h of patient’s data after ICU admission with at least 30 h of present data
        5. Split data into train/validation/test with 70%/10%/20% ratio
        6. Impute the admissions data as provided in MIMIC-Extract GitHub code (not described in the paper, but used in code)

    Arguments:
        admissions: raw admissions data
        patients: raw patient data

    Outputs: processed sample patient dataset, and split admissions and patient datasets

    Note: using a function instead of running these steps outside the function changes the full dataset memory requirements from 13GB RAM
    to 30GB RAM.
    """

    # 1. Subsample the data
    # subsampling to retain only data for the first n (sample size) unique subjects ids (patients) for demo purposes
    subsample_subject_id = admissions.index.get_level_values('subject_id').unique()[:sample_size]
    admissions = admissions[admissions.index.get_level_values('subject_id').isin(subsample_subject_id)]
    patients = patients[patients.index.get_level_values('subject_id').isin(subsample_subject_id)]
    # should produce 6682 rows vs 2200954
    print("\n Size/shape of sample admissions data: ", admissions.shape)

    # only consider patients with at least 30 hours of present data
    patients = patients[patients.max_hours > min_present_data][['mort_hosp', 'mort_icu', 'los_icu']]
    # create boolean labels for length of stay >3 and >7 targets
    patients['los_3'] = patients['los_icu'] > 3
    patients['los_7'] = patients['los_icu'] > 7
    # drop the original length of stay column
    patients.drop(columns=['los_icu'], inplace=True)
    patients.astype(float)

    # drop any ICU admissions that were removed from the patient data and where the hours in ICU are less than the window size
    # Overall this means only using the first 24 h of patient’s data after ICU admission for patients
    # with at least 30 hours of present data (as described in section 3.1. Data in the paper)
    admissions = admissions[
        (admissions.index.get_level_values('icustay_id').isin(set(patients.index.get_level_values('icustay_id')))) &
        (admissions.index.get_level_values('hours_in') < window_size)]

    # split dataset based on patients into 70% training, 10% validation and 20% testing sets
    train_split, val_split, test_split = 0.7, 0.1, 0.2

    ## make sure the 2 datasets have the same patients
    admissions_subj_idx,  patients_subj_idx = [df.index.get_level_values('subject_id') for df in (admissions, patients)]
    admissions_patients = set(admissions_subj_idx)
    assert admissions_patients == set(patients_subj_idx), "Subject ID pools differ!" # is this necessary??

    np.random.seed(SEED)
    all_patients, N = np.random.permutation(list(admissions_patients)), len(admissions_patients)
    N_train, N_val, N_test = int(train_split * N), int(val_split * N), int(test_split * N)
    train_subj_ids = all_patients[:N_train]
    val_subj_ids   = all_patients[N_train:N_train + N_val]
    test_subj_ids  = all_patients[N_train+N_val:]

    print(f"  Training split : {round(len(train_subj_ids)/N*100,1)}%")
    print(f"  Validation split : {round(len(val_subj_ids)/N*100,1)}%")
    print(f"  Testing split :: {round(len(test_subj_ids)/N*100,1)}%")

    [(admissions_train, admissions_val, admissions_test), (patients_train, patients_val, patients_test)] = [
        [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj_ids, val_subj_ids, test_subj_ids)] \
        for df in (admissions, patients)
    ]

    ### normalize the column means using the training dataset mean and standard deviation
    idx = pd.IndexSlice
    admissions_means_train = admissions_train.loc[:, idx[:,'mean']].mean(axis=0)
    admissions_stds_train = admissions_train.loc[:, idx[:,'mean']].std(axis=0)

    admissions_train.loc[:, idx[:,'mean']] = (admissions_train.loc[:, idx[:,'mean']] - admissions_means_train)/admissions_stds_train
    admissions_val.loc[:, idx[:,'mean']] = (admissions_val.loc[:, idx[:,'mean']] - admissions_means_train)/admissions_stds_train
    admissions_test.loc[:, idx[:,'mean']] = (admissions_test.loc[:, idx[:,'mean']] - admissions_means_train)/admissions_stds_train

    # impute the admissions data
    admissions_train, admissions_val, admissions_test = [
        simple_imputer(df) for df in (admissions_train, admissions_val, admissions_test)
    ]

    # check that there are no more nulls
    for df in admissions_train, admissions_val, admissions_test: assert not df.isnull().any().any()
    for df in patients_train, patients_val, patients_test: assert not df.isnull().any().any()

    # Split the patient data the same as admissions data (counts are more than reported, but the code seems ok)
    [(patients_train, patients_val, patients_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj_ids, val_subj_ids, test_subj_ids)] \
    for df in (patients,) ]

    return admissions_train, admissions_val, admissions_test, patients, patients_train, patients_val, patients_test



In [ ]:
## implement functions

# load full raw data
raw_admissions, raw_patients = load_raw_mimic_extract_data(raw_mimic_extract_data_dir)
# calculate and print statistics
calculate_stats(raw_admissions,raw_patients)
# process sample data
admissions_train, admissions_val, admissions_test, patients, patients_train, patients_val, patients_test = process_time_series_data(raw_admissions, raw_patients)

del raw_admissions
del raw_patients
gc.collect()


 Statistics of MIMIC-Extract raw data:
   -  Size/shape of dataset: (2200954, 312)
   -  Total # of patients: 34472
   -  Total # of hospital admissions: 34472
   -  Total # of ICU admissions: 34472
   -  Total # clinically aggregated time-series variables: 104
   - In-hospital mortality distribution: 9.6 %
   - In-ICU mortality distribution: 6.6 %
   - Length-of-stay > 3 distribution: 29.9 %
   - Length-of-stay > 7 distribution: 5.4 %
NOTE: the above 4 distributions do not all match the final cohort distribution.

 Size/shape of sample admissions data:  (6682, 312)
  Training split : 69.4%
  Validation split : 9.7%
  Testing split :: 20.8%


0

### MIMIC-III clinical notes



In [ ]:
# dir and function to load raw clinical notes data

raw_mimic_iii_noteevents_data_dir = f"{DATA_PATH}/NOTEEVENTS.csv"
raw_mimic_iii_admissions_data_dir = f"{DATA_PATH}/ADMISSIONS.csv" # only for statistics
raw_mimic_iii_icustays_data_dir = f"{DATA_PATH}/ICUSTAYS.csv" # only for statistics

def load_raw_mimic_iii_notes_data():
    # load ICU clinical note data
    noteevents_data = pd.read_csv(raw_mimic_iii_noteevents_data_dir,low_memory=False)
    # load hospital admission data (for statistics)
    admission_data  = pd.read_csv(raw_mimic_iii_admissions_data_dir,low_memory=False)
    # load ICU admission data (for statistics)
    icustays_data  = pd.read_csv(raw_mimic_iii_icustays_data_dir,low_memory=False)
    return noteevents_data , admission_data , icustays_data

# calculate statistics
def calculate_stats(admission_data,icustays_data,notes_data):
    # implement this function to calculate the statistics
    print("Statistics of MIMIC-III raw data:")
    print("   - ", "Size/shape of hospital admissions dataset:", admission_data.shape)
    print("   - ", "Size/shape of ICU admissions dataset:", icustays_data.shape)
    print("   - ","Total # of patients:", len(admission_data['SUBJECT_ID'].unique()))
    print("   - ","Total # of hospital admissions:", len(admission_data['HADM_ID'].unique()))
    print("   - ","Total # of ICU admissions:", len(icustays_data['ICUSTAY_ID'].unique()))
    print("   - ","Total # of in ICU patients:", len(icustays_data['SUBJECT_ID'].unique())) # less patients than paper states = hospital patients??
    print("   - ","Total # of in note categories:", len(list(notes_data['CATEGORY'].unique())))


# process raw data
def process_data(raw_noteevents_data,patient_ids):
    """
    Take a sample of the datasets and process as described in the paper:
        1. drop discharge summaries
        2. drop clinical notes without chart time information
        3. subsample the data by subject ids from processed time series data
        3. drop patients without any clinical notes in 24 h
        4. Preprocess notes with author-provided script (preprocess.py)
        5. Apply the clinical NER model to get medical entities
        6. Apply Word2Vec, FastText, and a concatenation of both word
            embeddings to the medical entities to create representations

    Arguments:
        raw_noteevents_data: raw clinical notes data
        patient_ids: patient ids from the time series cohort

    Outputs:
        word2vec_embeddings: patient medical entity Word2Vec embeddings dictionary
        fasttext_embeddings: patient medical entity FastText embeddings dictionary
        concatenated_embeddings: Combined Word2Vec and FastText embeddings dictionary
    """
    # drop discharge summaries
    noteevents_data = raw_noteevents_data[~(raw_noteevents_data['CATEGORY'] == 'Discharge summary')]
    # drop clinical notes without chart time information
    noteevents_data.dropna(subset=['CHARTTIME'],inplace=True)
    # subsample the data by subject ids from processed time series data
    noteevents_data = noteevents_data[noteevents_data['SUBJECT_ID'].isin(patient_ids)]

    # only keep patients with clinical notes within 24 h
    ## first join to patient-level data to get intime attribute (time patient entered ICU)
    patients = pd.read_hdf(raw_mimic_extract_data_dir, 'patients').reset_index()
    patients.rename(columns = {"subject_id": "SUBJECT_ID", "hadm_id": "HADM_ID"}, inplace=True)
    admission_notes_data = pd.merge(noteevents_data[['SUBJECT_ID','CHARTTIME', 'TEXT']],
                            patients[['SUBJECT_ID','HADM_ID','intime']],
                            on = ['SUBJECT_ID'],
                            how = 'left')
    admission_notes_data['CHARTTIME'] = pd.to_datetime(admission_notes_data['CHARTTIME'])
    admission_notes_data = admission_notes_data[((admission_notes_data['CHARTTIME']-admission_notes_data['intime']).dt.total_seconds()/(60*60))<window_size]

    del patients, noteevents_data

    # preprocess note text with author provided script (preprocess.py)
    preprocess_path = f"{PROJECT_PATH}preprocess.py"
    %run "{preprocess_path}"
    nltk.download('punkt')
    admission_notes_data['preprocessed_text'] = admission_notes_data['TEXT'].apply(getSentences)

    # apply med7 on the clinical note text (takes 2 full days on full data)
    med7 = spacy.load("en_core_med7_lg")
    ## --
    admission_notes_data['ner'] = None
    count = 0
    preprocessed_index = {}
    for i in admission_notes_data.itertuples():

        if count % 1000 == 0: # not very useful with small sample
            print("MED 7 record: ",count)

        ind = i.Index
        text = i.preprocessed_text

        all_pred = []
        for each_sent in text:
            try:
                doc = med7(each_sent)
                result = ([(ent.text, ent.label_) for ent in doc.ents])
                if len(result) == 0: continue   ##
                all_pred.append(result)
            except:
                print("error..")
                continue
        admission_notes_data.at[ind, 'ner'] = all_pred
        count += 1
    ## remove rows without medical entities
    admission_notes_data = admission_notes_data[admission_notes_data['ner'].str.len() != 0]

    # change the admission notes data to a dictionary with patient ids as keys and note text as values (list items)
    # to apply word embeddings
    med7_ner_dict = {}

    for ii in admission_notes_data.itertuples():
        p_id = ii.SUBJECT_ID
        ind = ii.Index
        try:
            new_ner = admission_notes_data.loc[ind].ner
        except:
            new_ner = []

        unique = set()
        new_temp = []
        for j in new_ner:
            for k in j:
                unique.add(k[0])
                new_temp.append(k)

        if p_id in med7_ner_dict:
            for i in new_temp:
                med7_ner_dict[p_id].append(i)
        else:
            med7_ner_dict[p_id] = new_temp

    del admission_notes_data

    # Apply word embeddings
    ## load word emdedding models
    w2vec = Word2Vec.load(f"{EMBEDDING_PATH}/word2vec.model")
    fasttext = FastText.load(f"{EMBEDDING_PATH}/fasttext.model")

    ## define mean function
    def mean(a):
        return sum(a) / len(a)

    for data in [med7_ner_dict]:
        # processing fasttext and word2vec together as they have the same words
        # this may have been different when the paper was published
        print("w2vec and fasttext embeddings starting..")
        word2vec_embeddings = {}
        fasttext_embeddings = {}
        concatenated_embeddings = {}
        for k,v in data.items():
            patient_temp_w2v, patient_temp_ftv = [], []
            for i in v:
                try:
                    patient_temp_w2v.append(w2vec.wv[i[0]])
                    patient_temp_ftv.append(fasttext.wv[i[0]])
                except:
                    avg_w2v, avg_ftv = [],[]
                    temp_w2v, temp_ftv= [], []
                    num = 0
                    # where there are multiple words, split the text, apply embedding to each word and take the average
                    if len(i[0].split(" ")) > 1:
                        for each_word in i[0].split(" "):
                            try:
                                temp_w2v, temp_ftv = w2vec.wv[each_word], fasttext.wv[each_word]
                                avg_w2v.append(temp_w2v)
                                avg_ftv.append(temp_ftv)
                                num += 1
                            except:
                                pass
                        if num == 0: continue
                        avg_w2v, avg_ftv = np.asarray(avg_w2v), np.asarray(avg_ftv)
                        t = np.asarray(list(map(mean, zip(*avg_w2v))))
                        patient_temp_w2v.append(t)
                        t = np.asarray(list(map(mean, zip(*avg_ftv))))
                        patient_temp_ftv.append(t)
            if len(patient_temp_w2v) == 0: continue # checking w2v only
            word2vec_embeddings[k] = patient_temp_w2v
            fasttext_embeddings[k] = patient_temp_ftv

        #############################################################################

        print("combined emdeddings starting..")
        # concatenate the Word2Vec and FastText representations horizontally (ci ∈ ℝ200).
        # add zero padding for missing word embeddings, authors only pad Word2vec, but fastext
        # and word2vec have the same number of words currently, so padding both.
        for k,v in data.items():
            patient_temp_concat = []
        #     if k != 6: continue
            for i in v:
                w2vec_temp, fasttext_temp = [], []
                try:
                    w2vec_temp, fasttext_temp = w2vec.wv[i[0]], fasttext.wv[i[0]]
                except:
                    avg_w2v, avg_ftv = [],[]
                    temp_w2v, temp_ftv= [], []
                    num = 0
                    # where there are mutliple words, split the text, apply embedding to each word and take the average
                    if len(i[0].split(" ")) > 1:
                        for each_word in i[0].split(" "):
                            try:
                                temp_w2v, temp_ftv = w2vec.wv[each_word], fasttext.wv[each_word]
                                avg_w2v.append(temp_w2v)
                                avg_ftv.append(temp_ftv)
                                num += 1
                            except:
                                pass
                        if num == 0:
                            w2vec_temp, fasttext_temp = [0] * 100, [0] * 100
                        else:
                            avg_w2v, avg_ftv = np.asarray(avg_w2v), np.asarray(avg_ftv)
                            w2vec_temp = np.asarray(list(map(mean, zip(*avg_w2v))))
                            fasttext_temp = np.asarray(list(map(mean, zip(*avg_ftv))))
                    else:
                        w2vec_temp, fasttext_temp = [0] * 100, [0] * 100

                appended = np.append(fasttext_temp, w2vec_temp, 0)
                patient_temp_concat.append(appended)

            if len(patient_temp_concat[0]) == 0: continue # if no embeddings, skip,
            concatenated_embeddings[k] = patient_temp_concat

        #print(len(word2vec_embeddings), len(fasttext_embeddings), len(concatenated_embeddings))

    # only keep embeddings that are present in word2vec
    diff = set(fasttext_embeddings.keys()).difference(set(word2vec_embeddings))
    for i in diff:
        del fasttext_embeddings[i]

    diff = set(concatenated_embeddings.keys()).difference(set(word2vec_embeddings))
    for i in diff:
        del concatenated_embeddings[i]

    print("word2vec patients: ",len(word2vec_embeddings))
    print("fasttext patients: ",len(fasttext_embeddings))
    print("combined patients: ",len(concatenated_embeddings))

    return word2vec_embeddings, fasttext_embeddings, concatenated_embeddings




In [ ]:
## implement functions
# Extract raw data and print statistics
raw_noteevents_data, raw_admission_data, raw_icustays_data = load_raw_mimic_iii_notes_data()
calculate_stats(raw_admission_data, raw_icustays_data,raw_noteevents_data)

del raw_admission_data, raw_icustays_data
gc.collect()

Statistics of MIMIC-III raw data:
   -  Size/shape of hospital admissions dataset: (58976, 19)
   -  Size/shape of ICU admissions dataset: (61532, 12)
   -  Total # of patients: 46520
   -  Total # of hospital admissions: 58976
   -  Total # of ICU admissions: 61532
   -  Total # of in ICU patients: 46476
   -  Total # of in note categories: 15


0

In [ ]:
## implement functions
# store subject/patient ids from processed time series data
subject_ids = list(patients.index.get_level_values('subject_id'))

# process data
word2vec_embeddings, fasttext_embeddings, concatenated_embeddings = process_data(raw_noteevents_data,subject_ids)

del raw_noteevents_data
del FastText, Word2Vec


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


MED 7 record:  0
w2vec and fasttext embeddings starting..
combined emdeddings starting..
word2vec patients:  72
fasttext patients:  72
combined patients:  72


### Final time series datasets

In [ ]:
# only keep time series patients with notes
patients_to_keep = word2vec_embeddings.keys()

train_patients = patients_train.index.get_level_values('subject_id').intersection(patients_to_keep)
val_patients = patients_val.index.get_level_values('subject_id').intersection(patients_to_keep)
test_patients = patients_test.index.get_level_values('subject_id').intersection(patients_to_keep)

## target
y_train = patients_train.loc[train_patients]
y_val = patients_val.loc[val_patients]
y_test = patients_test.loc[test_patients]

## variables
admissions_train = admissions_train.loc[train_patients]
admissions_val = admissions_val.loc[val_patients]
admissions_test = admissions_test.loc[test_patients]

### only retain the hourly mean sub columns for each measurement column
admissions_train = admissions_train.loc[:, pd.IndexSlice[:, 'mean']]
admissions_val = admissions_val.loc[:, pd.IndexSlice[:, 'mean']]
admissions_test = admissions_test.loc[:, pd.IndexSlice[:, 'mean']]

### get the matrix representation of the admissions data
x_train_ts = admissions_train.values
x_val_ts = admissions_val.values
x_test_ts = admissions_test.values

del admissions_train, admissions_val, admissions_test
gc.collect()

### reshape the data for time series prediction
x_train_ts = x_train_ts.reshape(int(x_train_ts.shape[0] / 24), 24, 104)
x_val_ts = x_val_ts.reshape(int(x_val_ts.shape[0] / 24), 24, 104)
x_test_ts = x_test_ts.reshape(int(x_test_ts.shape[0] / 24), 24, 104)

In [ ]:
# If full datasets were processed, save for experiments and ablations
if sample_size == None:
    pd.to_pickle(x_train_ts, f"{DATA_PATH}/full_x_train.pkl")
    pd.to_pickle(x_val_ts, f"{DATA_PATH}/full_x_val.pkl")
    pd.to_pickle(x_test_ts, f"{DATA_PATH}/full_x_test.pkl")

    pd.to_pickle(y_train, f"{DATA_PATH}/full_y_train.pkl")
    pd.to_pickle(y_val, f"{DATA_PATH}/full_y_val.pkl")
    pd.to_pickle(y_test, f"{DATA_PATH}/full_y_test.pkl")

    pd.to_pickle(train_patients, f"{DATA_PATH}/full_train_ids.pkl")
    pd.to_pickle(val_patients, f"{DATA_PATH}/full_dev_ids.pkl")
    pd.to_pickle(test_patients, f"{DATA_PATH}/full_test_ids.pkl")

    pd.to_pickle(word2vec_embeddings, f"{DATA_PATH}/full_word2vec_embeddings.pkl")
    pd.to_pickle(fasttext_embeddings, f"{DATA_PATH}/full_fasttext_embeddings.pkl")
    pd.to_pickle(concatenated_embeddings, f"{DATA_PATH}/full_concat_embeddings.pkl")


##   Model
The section includes the proposed model definition function, model training and evaluation of the subsampled processed data.

-------------------------------

**Citation to the original paper:**
Bardak, B., & Tan, M. (2021). Improving clinical outcome predictions using convolution over medical entities with multimodal learning. Artificial intelligence in medicine, 117, 102112. https://doi.org/10.1016/j.artmed.2021.102112

**Original papers GitHub link:** https://github.com/tanlab/ConvolutionMedicalNer

---

The proposed model is a convolutional-based multimodal deep learning architecture. This model is implemented using Keras.

**Model architecture:** (layer number/size/type, activation function, etc)

-	A 3-layer 1D Convolutional Neural Network (CNN), with filter sizes 32, 64, and 96 respectively, Relu activation and a max pooling layer to extract features from the medical entities
- A single-layer Gated Recurrent Unit (GRU) with
256 hidden units to extract features from the time series data.
-	Lastly one fully connected layer with 256 hidden units and Relu activation, that takes the concatenated outputs of above layers to make predictions.


**Training objectives:** (loss function, optimizer, weight of each loss term, etc)

- 0.2 dropout rate is used at the end of the fully connected layer
- L2 norm for sparsity regularization is selected with A 0.01 scale factor.
- ADAM algorithm with a learning rate of 0.001 is used for optimization.
- All models are trained to minimize the binary cross-entropy loss.
- Batch size of 64 is used for training. (not specified in paper, taken from code)

**Evaluation metrics:**

Three different metrics are used due to class imbalances; AUROC, AUPRC and F1.

1. AUROC (area under the Receiver Operating Characteristic curve) is a commonly used metric that is robust to imbalanced data.
2. AUPRC (area under the precision-recall curve) is similar to AUROC but does not consider true negatives. Since the data contains a lot of true negatives, this metric is a meaningful measure of success.
3. F1 combines precision and recall while giving equal importance to false negatives and positives.

For all three metrics, the higher the score, the better the model.

Reference: https://link.springer.com/chapter/10.1007/978-3-030-82184-5_3#Sec10
(see subsection 3.4.2.2 Real-Value Prediction for Classification)

---

**Computational requirements:** The full proposed model successfully runs in free Google Colab both under CPU and GPU (as initially predicted in the project proposal). The model trained using the full dataset runs for:
- about 5-6 hours with 13 GB CPU RAM (each epoch running on average for 25 seconds) and,
- about 1 hour 42 mins on 15GB GPU RAM (each epoch running on average for 7 seconds).

The model is trained for 50 epochs for each embedding and prediction problem, with early stopping after 3 epochs with no change in validation loss (most iterations ran for between 5-7 epochs). Each model was trained 10 times with different initialization seeds.

**NOTE:** the pretrained models (trained outside this notebook on the full dataset) are loaded and tested under results section.










### Parameters

In [ ]:
embedding_types = ['word2vec', 'fasttext', 'concat']
embedding_dict = [word2vec_embeddings, fasttext_embeddings, concatenated_embeddings]
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']

# parameters/inputs
num_epoch = 3 # for demo (original in paper = 50)
model_patience = 3
monitor_criteria = 'val_loss'
ner_representation_limit = 64 # size of patient input matrix

## Hyperparameters
batch_size = 64 # not specified in paper
ts_hidden_unit = 256
filter_sizes = [32,64,96]
# the rest are specified directly in the model function.

maxiter = 2 # 1 iteration for demo (original = 10)

### Define Proposed Model




In [ ]:
# define model
def proposed_model(number_of_unit, embedding_name, ner_limit, num_filter):
    """
    Arguments:
        number_of_unit: GRU number of hidden units
        embedding_name: name of embedding
        ner_limit: size of patient matrix
        num_filter: list of 3 CNN filter sizes

    Outputs:
        model: the proposed multimodal model
    """
    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    # 24 hours and 104 time-series variable
    ts_input = Input(shape=(24,104),  name = "timeseries_input")
    input_embeddings = Input(shape=(ner_limit, input_dimension), name = "cnn_input")


    # 3 1D convolutional layers, with no padding
    text_conv1d = Conv1D(filters=num_filter[0], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                         kernel_initializer=GlorotUniform() )(input_embeddings)

    text_conv1d = Conv1D(filters=num_filter[1], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=GlorotUniform())(text_conv1d)

    text_conv1d = Conv1D(filters=num_filter[2], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=GlorotUniform())(text_conv1d)

    # max pooling layer
    text_embeddings = GlobalMaxPooling1D()(text_conv1d)

    # GRU layer for time series data
    x = GRU(number_of_unit)(ts_input)

    # concatenate time series features and medical entity feature
    concatenated = Concatenate(axis=1)([x, text_embeddings])

    # Fully connected layer
    concatenated = Dense(512, activation='relu')(concatenated)
    concatenated = Dropout(0.2)(concatenated)

    preds = Dense(1, activation='sigmoid',use_bias=False,
                          kernel_initializer=GlorotUniform(),
                  kernel_regularizer=L2(l2=0.01))(concatenated)

    model = Model(inputs=[ts_input, input_embeddings], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['acc'])

    return model

### Evaluation

In [ ]:
def print_scores(model, test_data, ground_truth):
    """
    Prints auc, auprc and f1 scores to the screen.

    Arguments:
        model: the proposed model
        test_data: testing data
        ground_truth: the proposed model

    Outputs:
        None
    """
    # make predictions
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]

    #evaluate
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, y_pred)
    F1    = f1_score(ground_truth, y_pred)

    print ("AUC: ", auc, "AUPRC: ", auprc, "F1: ", F1)

### Train and evaluate Model


In [ ]:
%%time

def get_subvector_data(size, embed_name, data):
    """
    Turn embeddings into matrix of the dimensions vector_size x size
    """
    # specify vector size
    if embed_name == "concat":
        vector_size = 200
    else:
        vector_size = 100

    x_data = {}
    for k, v in data.items():
        # get the number of additional vectors needed for padding
        number_of_additional_vectors = len(v) - size
        vector = []
        for i in v:
            vector.append(i)
        # If vector length is less than required size, pad with zero vectors
        # else, slice the vector to keep only the required size
        if number_of_additional_vectors < 0:
            number_of_additional_vectors = np.abs(number_of_additional_vectors)

            temp = vector[:size]
            for i in range(0, number_of_additional_vectors):
                temp.append(np.zeros(vector_size))
            x_data[k] = np.asarray(temp)
        else:
            x_data[k] = np.asarray(vector[:size])

    return x_data


## training

print("""-*--------------*-----------------*-----

TRAINING ON SUBSET FOR DEMO PURPOSES ONLY

----------*--------------*-------------------*----""")

for embed_dict, embed_name in zip(embedding_dict, embedding_types):
    print ("Embedding: ", embed_name)
    print("=============================")

    # split embeddings into training, validation and testing
    temp_train_ner = dict((k, embed_dict[k]) for k in train_patients)
    tem_val_ner = dict((k, embed_dict[k]) for k in val_patients)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_patients)

    # combine medical entiy embeddings vertically, with padding, to create a patient matrix representation
    x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
    x_val_dict = get_subvector_data(ner_representation_limit, embed_name, tem_val_ner)
    x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

    x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
    x_val_dict_sorted = collections.OrderedDict(sorted(x_val_dict.items()))
    x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))

    x_train_ner = np.asarray(list(x_train_dict_sorted.values()))
    x_val_ner = np.asarray(list(x_val_dict_sorted.values()))
    x_test_ner = np.asarray(list(x_test_dict_sorted.values()))

    # train model for each iteration and each problem
    for iteration in range(1,maxiter):
        print ("Iteration number: ", iteration)

        #set a different seed for each iteration (still get slightly different results with each run)
        set_seed(SEED + iteration)

        for each_problem in target_problems:
            print ("__________________")
            print ("Problem type: ", each_problem)

            # employ early stopping
            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
            callbacks = [early_stopping_monitor]

            # load and train model
            model = proposed_model(ts_hidden_unit,
                               embed_name, ner_representation_limit,filter_sizes)
            model.fit([x_train_ts, x_train_ner], y_train[each_problem],
                      epochs=num_epoch, verbose=1,
                      validation_data=([x_val_ts, x_val_ner], y_val[each_problem]),
                      callbacks=callbacks, batch_size=batch_size)

            # make predictions and evaluate
            print_scores(model, [x_test_ts, x_test_ner], y_test[each_problem])

            del model
            clear_session()
            gc.collect()

-*--------------*-----------------*-----

TRAINING ON SUBSET FOR DEMO PURPOSES ONLY

----------*--------------*-------------------*----
Embedding:  word2vec
Iteration number:  1
__________________
Problem type:  mort_hosp
Train on 50 samples, validate on 7 samples
Epoch 1/3
50/50 [==============================] - 1s 14ms/sample - loss: 0.5210 - acc: 0.8600 - val_loss: 0.6470 - val_acc: 0.8571
Epoch 2/3
50/50 [==============================] - 0s 2ms/sample - loss: 0.3104 - acc: 0.9400 - val_loss: 0.8141 - val_acc: 0.8571
Epoch 3/3
50/50 [==============================] - 0s 2ms/sample - loss: 0.2726 - acc: 0.9400 - val_loss: 0.8127 - val_acc: 0.8571
AUC:  0.27777777777777773 AUPRC:  0.18888888888888888 F1:  0.0
__________________
Problem type:  mort_icu
Train on 50 samples, validate on 7 samples
Epoch 1/3
50/50 [==============================] - 1s 11ms/sample - loss: 0.5742 - acc: 0.7800 - val_loss: 0.6571 - val_acc: 0.8571
Epoch 2/3
50/50 [==============================] - 0s 2ms/sa

In [ ]:
del concatenated_embeddings, val_patients, embed_dict, embed_name, embedding_dict
del patients,patients_val, patients_test, patients_to_keep, patients_train, tem_val_ner
del temp_test_ner,temp_train_ner,test_patients,word2vec_embeddings
del x_val_dict,x_val_dict_sorted,x_val_ner,x_val_ts,x_test_dict,x_test_dict_sorted
del x_test_ner,x_test_ts,x_train_dict,x_train_dict_sorted,x_train_ner,x_train_ts,
del y_val, y_test,y_train
gc.collect()

0

In [ ]:
# Checking that the whole notebook does not exceed 8 mins (not hard rule but needs to run fast))
print("Total running time = {:.2f} minutes".format((time.time() - _START_RUNTIME)/60))

Total running time = 6.24 minutes


# Results
This section includes:

1. For demo purposes, the pretrained models (trained outside this notebook) are loaded and tested. These models are the best models from the last (10th) iteration.
2. Table of results for the proposed model (average of all 10 iterations)
    - All claims are supported by experiment results
    - Discussion with respect to the hypothesis and results from the original paper
3. Ablation Study.

The results are all reported with AUROC (AUC), AUPRC and F1.

## Proposed Model

### Pretrained model and preprocessed data

In [ ]:

def load_processed_data(raw_data_dir):
    """load the full processed testing data"""
    # load time series data
    x_test_ts= pd.read_pickle(f"{raw_data_dir}/full_x_test.pkl")
    y_test = pd.read_pickle(f"{raw_data_dir}/full_y_test.pkl")

    test_patients = y_test.index.get_level_values('subject_id')

    # loading full embeddings, but only keeping test patients
    word2vec_embeddings_test = pd.read_pickle(f"{raw_data_dir}/full_word2vec_embeddings.pkl")
    word2vec_embeddings_test = dict((k, word2vec_embeddings_test[k]) for k in test_patients)

    fasttext_embeddings_test = pd.read_pickle(f"{raw_data_dir}/full_fasttext_embeddings.pkl")
    fasttext_embeddings_test = dict((k, fasttext_embeddings_test[k]) for k in test_patients)

    concat_embeddings_test = pd.read_pickle(f"{raw_data_dir}/full_concat_embeddings.pkl")
    concat_embeddings_test = dict((k, concat_embeddings_test[k]) for k in test_patients)

    return x_test_ts, y_test, word2vec_embeddings_test, fasttext_embeddings_test , concat_embeddings_test


# load processed data
x_test_ts, y_test, word2vec_embeddings_test, fasttext_embeddings_test, concat_embeddings_test = load_processed_data(DATA_PATH)


In [ ]:
embedding_types = ['word2vec', 'fasttext', 'concat']
embedding_dict = [word2vec_embeddings_test, fasttext_embeddings_test, concat_embeddings_test]
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
iteration = 0 # only saved the best model model from the last iteration

# load best 10nth iteration pretrained models and evaluate on testing data
for embed_dict, embed_name in zip(embedding_dict, embedding_types):
    print ("\n Embedding: ", embed_name)
    print("=============================")

    x_test_dict = get_subvector_data(ner_representation_limit, embed_name, embed_dict)
    x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))
    x_test_ner = np.asarray(list(x_test_dict_sorted.values()))

    for each_problem in target_problems:
        print ("__________________")
        print ("Problem type: ", each_problem)

        # create a model instance
        pretrained_model = proposed_model(ts_hidden_unit,
                            embed_name, ner_representation_limit,filter_sizes)

        # Load the weights
        pretrained_model.load_weights(f"{RESULTS_PATH}/64-basiccnn1d-{embed_name}-{each_problem}-best_model.hdf5")

        # print metrics to evaluate model
        print_scores(pretrained_model, [x_test_ts, x_test_ner], y_test[each_problem])


 Embedding:  word2vec
__________________
Problem type:  mort_hosp
AUC:  0.8753810934554268 AUPRC:  0.5644586051009224 F1:  0.4621733149931225
__________________
Problem type:  mort_icu
AUC:  0.8803528881447615 AUPRC:  0.5191799884847719 F1:  0.4733475479744136
__________________
Problem type:  los_3
AUC:  0.7064488719993578 AUPRC:  0.6464571955831138 F1:  0.5618848792126454
__________________
Problem type:  los_7
AUC:  0.730345793252078 AUPRC:  0.22082511832769958 F1:  0.02197802197802198

 Embedding:  fasttext
__________________
Problem type:  mort_hosp
AUC:  0.8714225693509292 AUPRC:  0.5524502560820251 F1:  0.46433566433566437
__________________
Problem type:  mort_icu
AUC:  0.8781734021446816 AUPRC:  0.5074139181918761 F1:  0.4562899786780384
__________________
Problem type:  los_3
AUC:  0.6996246688254342 AUPRC:  0.6408740851149082 F1:  0.5552256532066507
__________________
Problem type:  los_7
AUC:  0.7257790637790927 AUPRC:  0.2059790210232904 F1:  0.011111111111111112

 Embedd

### Table of results and comparison

In [6]:
# load the saved original paper results as reported
reported_results = pd.read_csv(f"{RESULTS_PATH}/reported_results.csv")

#display(reported_results.head(12))

# load the saved experiment results for all models and iterations
full_results = pd.read_csv(f"{RESULTS_PATH}/results.csv")
# printing the word2vec results from the 10nth iteration as they match above model results
full_results[(full_results['Model'] == 'Proposed') & (full_results['Embedding'] == 'word2vec') & (full_results['Iteration'] == 10)]

,Task,Model,Embedding,Iteration,AUC,AUPRC,ACC,F1
36,mort_hosp,Proposed,word2vec,10,0.875381,0.564459,0.911538,0.462173
37,mort_icu,Proposed,word2vec,10,0.880353,0.519180,0.944118,0.473348
38,los_3,Proposed,word2vec,10,0.706449,0.646457,0.667647,0.561885
39,los_7,Proposed,word2vec,10,0.730346,0.220825,0.919457,0.021978


In [7]:
# Replace tasks as in paper
task_map = {
    "los_3": "LOS >3 days",
    "los_7": "LOS >7 days",
    "mort_hosp": "In-hospital mortality",
    "mort_icu": "In-ICU mortality"
}
full_results["Task"] = full_results["Task"].map(task_map)
full_results['Embedding'] = full_results['Embedding'].replace('', np.nan).fillna('-')
reported_results["Task"] = reported_results["Task"].map(task_map)

# save the standard deviation for explanation
results_std = full_results.groupby(['Task', 'Model', 'Embedding']).agg(AUC_std=('AUC', 'std'),
                                                                       AUPRC_std=('AUPRC', 'std'),
                                                                       F1_std=('F1', 'std')).round(4)*100
results_std.reset_index(inplace=True)

# average the results from the 10 iterations

full_results.drop(columns=['Iteration','ACC'],inplace=True)
full_results = full_results.groupby(['Task', 'Model', 'Embedding']).mean().round(4)*100
full_results.reset_index(inplace=True)

# only retain the proposed model results
proposed_model_results = full_results[full_results['Model'] == 'Proposed']
#proposed_model_results

#### Scores

In [8]:
# Compare and display table of results
proposed_model_comparison = pd.merge(proposed_model_results.drop(columns=['Model']),
                                     reported_results[reported_results['Model'] == 'Proposed (Reported)'].drop(columns=['Model','AUC_std','AUPRC_std','F1_std']),
                                     on=['Task','Embedding'], suffixes=('', '_reported'))

## calculate the difference in scores
proposed_model_comparison['AUC_diff'] = proposed_model_comparison['AUC'] - proposed_model_comparison['AUC_reported']
proposed_model_comparison['AUPRC_diff'] = proposed_model_comparison['AUPRC'] - proposed_model_comparison['AUPRC_reported']
proposed_model_comparison['F1_diff'] = proposed_model_comparison['F1'] - proposed_model_comparison['F1_reported']
proposed_model_comparison = proposed_model_comparison[['Task','Embedding','AUC','AUC_reported','AUC_diff','AUPRC',
                                                      'AUPRC_reported','AUPRC_diff','F1','F1_reported','F1_diff']]
proposed_model_comparison

,Task,Embedding,AUC,AUC_reported,AUC_diff,AUPRC,AUPRC_reported,AUPRC_diff,F1,F1_reported,F1_diff
0,In-ICU mortality,concat,87.79,87.66,0.13,50.37,48.74,1.63,40.50,42.24,-1.74
1,In-ICU mortality,fasttext,87.19,87.85,-0.66,49.70,48.78,0.92,39.16,43.09,-3.93
2,In-ICU mortality,word2vec,87.47,88.35,-0.88,50.23,49.23,1.00,40.97,43.02,-2.05
3,In-hospital mortality,concat,87.43,86.98,0.45,56.65,55.35,1.30,45.17,46.38,-1.21
4,In-hospital mortality,fasttext,87.15,87.15,0.00,56.06,55.68,0.38,44.79,46.87,-2.08
5,In-hospital mortality,word2vec,87.46,87.55,-0.09,56.89,55.87,1.02,45.42,47.23,-1.81
6,LOS >3 days,concat,70.03,69.93,0.10,63.95,62.77,1.18,55.71,55.82,-0.11
7,LOS >3 days,fasttext,69.63,69.64,-0.01,63.70,62.55,1.15,54.46,55.87,-1.41
8,LOS >3 days,word2vec,70.06,69.54,0.52,64.23,62.68,1.55,55.21,55.82,-0.61
9,LOS >7 days,concat,71.38,71.92,-0.54,20.70,18.25,2.45,3.37,1.38,1.99


#### Standard Deviation

In [9]:
proposed_model_comparison = pd.merge(results_std[(results_std['Model'] == 'Proposed')].drop(columns=['Model']),
                                     reported_results[reported_results['Model'] == 'Proposed (Reported)'].drop(columns=['Model','AUC','AUPRC','F1']),
                                     on=['Task','Embedding'], suffixes=('', '_reported'))
proposed_model_comparison = proposed_model_comparison[['Task','Embedding','AUC_std','AUC_std_reported','AUPRC_std',
                                                      'AUPRC_std_reported','F1_std','F1_std_reported']]
proposed_model_comparison

,Task,Embedding,AUC_std,AUC_std_reported,AUPRC_std,AUPRC_std_reported,F1_std,F1_std_reported
0,In-ICU mortality,concat,0.71,0.002,1.48,0.009,5.81,0.027
1,In-ICU mortality,fasttext,0.57,0.001,1.30,0.009,4.03,0.026
2,In-ICU mortality,word2vec,0.51,0.002,1.43,0.008,4.26,0.029
3,In-hospital mortality,concat,0.44,0.003,0.79,0.008,1.83,0.027
4,In-hospital mortality,fasttext,0.44,0.002,0.94,0.005,2.37,0.015
5,In-hospital mortality,word2vec,0.57,0.003,0.69,0.008,1.62,0.014
6,LOS >3 days,concat,0.46,0.001,0.59,0.002,2.60,0.008
7,LOS >3 days,fasttext,0.44,0.003,0.75,0.003,2.48,0.017
8,LOS >3 days,word2vec,0.63,0.002,0.94,0.003,2.71,0.012
9,LOS >7 days,concat,1.54,0.007,0.96,0.006,4.39,0.009


### Discussion: Original paper result comparison

**NOTE:** results with respect to the hypothesis will be discussed under ablations.

1. For AUC, we reproduced scores to be within 1% of the reported values.
2. The AUPRC scores were all higher than the reported values within 3%, with LOS > 7 days having the largest deviation.
2. All F1 scores, except for LOS > 7 days are lower than the reported value within 4%. LOS > 7 days has a higher F1 score than reported, within 2%.

Based on AUC and AUPRC, the results uphold the paper’s conclusion that the proposed model performs better than the reported baseline for these metrics. However, experiment F1 scores refute the paper's conclusion on all F1 scores except LOS > 7 days.

However, the standard deviations of our experiments are much higher than the paper. The experiment standard deviations were between 0.44% and 5.81% while the paper reported between 0.001% and 0.029% standard deviation. Most reported results are within the standard deviation of the experiment results, so it is possible to get the same results as reported, by chance.

## Ablation Study

Ablations are done based on the hypothesis:
- Model trained without time series/ with medical entities only (assess the effect of multimodal approach)
- Model trained without medical entities/with time series only (assess the effect of multimodal approach)
- Model trained without convolution on medical entities (assess the effect of the convolution layer)

The ablations are geared at isolating the effect of the multimodal approach and the different techniques (word embeddings and convolution) on the unstructured clinical notes in the final proposed model.

See notebook: "Ablations.ipynb"

The averaged results for the 10 iterations are displayed.

In [8]:
def highlight_max_group(group):
    """highlight the maximum score in the group"""
    return ['background-color: orange' if v == group.max() else '' for v in group]

### Effect of Convolution

To assess the effect of the convolutional layers, we remove these layers and average the embeddings instead. This model is referred to as *No Convolution*.

After removing convolution, the model performed better on all scores except length of stay AUPRC (see table below, the best scores are highlighted in orange).

This challenges the effectiveness of the convolution, and the papers claims that the proposed model outperforms a multimodal approach without convolution.

In [12]:
# Initialize HTML string
html_output = "<table style='border-collapse: collapse; border: 1px solid black;'>"

# Group by Task and apply highlighting, then add to the HTML output
for i, (name, group) in enumerate(full_results.groupby("Task")):
    group.drop(group.columns[0], axis=1, inplace=True)
    group = group[group['Model'].isin(['No Convolution','Proposed'])]
    styled_group = group.style.apply(highlight_max_group, subset=["AUC", "AUPRC", "F1"])
    # format scores back to 2 decimal places and hide index
    styled_group = styled_group.format({col: "{:.4}" for col in ["AUC", "AUPRC", "F1"]})
    styled_group = styled_group.hide(axis="index")
    # Add title as task name
    html_output += f"<td colspan='2' style='border: 1px solid black; text-align: center; padding: 5px; font-weight: bold;'>{name}</td>"
    # Convert styled DataFrame to HTML and add to the output
    html_output += f"<td style='border: 1px solid black; text-align: center; padding: 8px;line-height: 20px;'>{styled_group.to_html()}</td>"
    # Display only two tables in a row
    if (i + 1) % 2 == 0:
        html_output += "</tr><tr>"

# Close table
html_output += "</table>"
display(HTML(html_output))

In [ ]:
# # Group by Task and apply the highlighting without html
# for name, group in full_results.groupby("Task"):
#     print ("\n\n Task: ", name)
#     print("=============================")
#     group.drop(group.columns[0], axis=1, inplace=True)
#     group = group[group['Model'].isin(['No convolution','Proposed'])]
#     styled_group = group.style.apply(highlight_max_group, subset=["AUC", "AUPRC", "F1"])
#     # format scores back to 2 decimal places and hide index
#     styled_group = styled_group.format({col: "{:.4}" for col in ["AUC", "AUPRC", "F1"]})
#     styled_group = styled_group.hide(axis="index")
#     display(styled_group)

### Effect of Multimodal Approach

To assess the effect of the multimodal approach, we further removed medical entities (refered to as *Time Series* model) and the time series features (refered to as *No Time Series* model, which contains averaged embeddings) respectively. This gives two unimodal models.

The *No Medical Entities* model performed better on all length of stay > 7 days scores, and on mortality AUCs. Thus effectiveness of the medical entities can be verified for most tasks, but not as many as the paper reported.

The *No Time Series* model deteriorates performance significantly (by at least 7%). This also further verifies the effectiveness of the multimodal approach for most tasks.

In [9]:
# Initialize HTML string
html_output = "<table style='border-collapse: collapse; border: 1px solid black;'>"

# Group by Task and apply highlighting, then add to the HTML output
for i, (name, group) in enumerate(full_results.groupby("Task")):
    group.drop(group.columns[0], axis=1, inplace=True)
    group = group[~group['Model'].isin(["Proposed (Tuned)"])]
    styled_group = group.style.apply(highlight_max_group, subset=["AUC", "AUPRC", "F1"])
    # format scores back to 2 decimal places and hide index
    styled_group = styled_group.format({col: "{:.4}" for col in ["AUC", "AUPRC", "F1"]})
    styled_group = styled_group.hide(axis="index")
    # Add title as task name
    html_output += f"<td colspan='2' style='border: 1px solid black; text-align: center; padding: 5px; font-weight: bold;'>{name}</td>"
    # Convert styled DataFrame to HTML and add to the output
    html_output += f"<td style='border: 1px solid black; text-align: center; padding: 8px;line-height: 20px;'>{styled_group.to_html()}</td>"
    # Display only two tables in a row
    if (i + 1) % 2 == 0:
        html_output += "</tr><tr>"

# Close table
html_output += "</table>"
display(HTML(html_output))


## Additional Experiments

### Hyperparameter tuning
Section 4.1 from the paper states that the authors independently tuned hyperparameters for all models. The proposed model at the beginning of this section uses the hyperparameter values stated in the paper, however, tuning hyperparameters for our current environment may improve performance and perhaps standard deviation as well.

The following hyperparameters are tuned on the validation set only using Word2Vec and In hospital mortality for 1 iteration and 5 epochs:
- number of hidden untis (original = 256, best = no change)
- convolutional filters (original = [32, 64, 96], best = [64, 96, 128])
- learning rate (original = 0.001, best = no change)
- dropout rates (original = 0.2, best = 0.5)
- regularization parameters (original = 0.01, best = 0.1)

Not all hyperparameters were tuned and smaller ranges were used due to time constraints. The tuned model results are also only abvailable for Word2vec and FastText due to usage limits in colab as this was the last model run.

The tuned model outperformed the proposed model, except for F1 scores for in-ICU mortality and length of stay > 7 days. However the standard deviation was still high. The results of the model with these hyperparameters are below.

Note: did not include ablations as results are not comparable due to differences in paramters, however, doing ablations on this model would be an interesting experiment to compare the effects.

In [13]:
# Initialize HTML string
html_output = "<table style='border-collapse: collapse; border: 1px solid black;'>"

# Group by Task and apply highlighting, then add to the HTML output
for i, (name, group) in enumerate(full_results.groupby("Task")):
    group.drop(group.columns[0], axis=1, inplace=True)
    group = group[group['Model'].isin(['Proposed', "Proposed (Tuned)"])]
    group = group[group['Embedding'].isin(['word2vec','fasttext'])]
    styled_group = group.style.apply(highlight_max_group, subset=["AUC", "AUPRC", "F1"])
    # format scores back to 2 decimal places and hide index
    styled_group = styled_group.format({col: "{:.4}" for col in ["AUC", "AUPRC", "F1"]})
    styled_group = styled_group.hide(axis="index")
    # Add title as task name
    html_output += f"<td colspan='2' style='border: 1px solid black; text-align: center; padding: 5px; font-weight: bold;'>{name}</td>"
    # Convert styled DataFrame to HTML and add to the output
    html_output += f"<td style='border: 1px solid black; text-align: center; padding: 8px;line-height: 20px;'>{styled_group.to_html()}</td>"
    # Display only two tables in a row
    if (i + 1) % 2 == 0:
        html_output += "</tr><tr>"

# Close table
html_output += "</table>"
display(HTML(html_output))

In [14]:
results_std[(results_std['Model'] == 'Proposed (Tuned)')]

,Task,Model,Embedding,AUC_std,AUPRC_std,F1_std
9,In-ICU mortality,Proposed (Tuned),fasttext,0.30,0.88,5.12
10,In-ICU mortality,Proposed (Tuned),word2vec,0.41,0.89,6.15
21,In-hospital mortality,Proposed (Tuned),fasttext,0.34,0.89,1.91
22,In-hospital mortality,Proposed (Tuned),word2vec,0.37,0.84,1.68
33,LOS >3 days,Proposed (Tuned),fasttext,0.46,0.45,2.31
34,LOS >3 days,Proposed (Tuned),word2vec,0.29,0.28,2.68
45,LOS >7 days,Proposed (Tuned),fasttext,0.91,0.90,0.87
46,LOS >7 days,Proposed (Tuned),word2vec,0.91,1.02,0.54


# Discussion

This section provides an assessment of the work done and recommendations to the authors.

## Assessment

The paper was partially reproducible. Only the experiment AUC scores were close to the reported scores. Ablation studies also showed that convolution did not improve results for most tasks as hypothesized in the paper, but improvement from the multimodal effect was confirmed for most tasks. **Due to the high standard deviation in the resukts, it is possible for someone to get the same results as the authors.**

### Reproduction efforts
* Used the author's code, with modifications for efficiency and fixes.
* Used the parameters specified from the paper and the code where unspecified.
* Split the data as the paper specified and created the cohort as described as well.

### Factors that made the paper irreproducible
* Differences in package versions as the requirements file was not provided. The code is 4 years old and some library methods were depricated.
* The final cohort was 22,203 patients compared to the reported 21,080. This difference may be due to changes in the MIMIC datasets. The paper was also missing a full/exact description of how clinical notes were preprocessed.
* Possible changes to word embeddings. Most noticeably: from the papers code, FastText seemed to have more words than Word2Vec. However, the experiement revealed that they has the same words, which resulted in the code failing during training.
* The paper was missing some parameters and hyperparameters like batch size, patient matrix column dimension (size parameter for the get_subvector_data function), kernel size, and early stopping number. (with more time, thorough hyperparameter tuning may have been helpful)
* Despite setting seeds, the standard deviations of the model results were high, expecially on GPU vs CPU. Rerunning the model could give slightly different results even when averaging.

### Communication with original authors
I reached out to one of the original authors, Batuhan Bardak. I shared my results, explained that the packages may be the biggest difference and asked for a requirements file. They could not provide one as the code is old, but they were kind enough to offer other possible reasons for the discrepancies quoted below:

"I believe this issue could be due to the following reasons:

- changes in the MIMIC-III dataset (in terms of the number of samples or attributes)
- differences in library version (as you mentioned)
- differences in the word embeddings
- General randomness (library, hardware, model, weight initialization etc.)
"

## What was easy
- The author's code in their GitHub repository made reproducing the experiments easier. It offered a very good starting point and had all the experiements in the paper. The first half of the author's code was generally easy to run. Specifically:

    - The original time series data processing code (01-Extract-Timeseries-Features.ipynb) ran well with no errors except for a memory error when reading the data. These memory errors were resolved by modifying the code to run within the memory constraints by only reading the admissions file once instead of twice as the authors did. However, when running with 24 GB RAM, no modifications are needed.
    - The code to process the clinical notes up to the application of NER ran well with no errors (02-Select-SubClinicalNotes.ipynb, 03-Preprocess-Clinical-Notes.ipynb, 04-Apply-med7-on-Clinical-Notes.ipynb).

- The main idea and method the paper used were very well explained and did not require advanced knowledge to follow.
- Not having to run the MIMIC-Extract code as the preprocessed data was saved in Google Cloud Platform. From the GitHub page execution " Will probably take 5-10 hours. Will require a good machine with at least 50GB RAM."


## What was difficult
- The major difficulty was from the lack of a requirements file. A lot of libraries and methods were deprecated or methods were moved to different modules so the code needed to be changed (particularly the model code). This may not prove as difficult for someone familiar with the Keras.
- The data processing code took 3+ days to run on the full dataset. The med7 application particularly took the longest (about 15 hours). With Google Colabs 12 hour limit, modifications needed to be made to run it in chunks.
- There were extra preprocessing steps in the code that were commented out and seemed unused. I would have liked to apply the extra steps to see if the results changed, but unfortunately that would take too long as mentioned in the previous point.
- The script to represent medical entities with word embeddings (05-Represent-Entities-With-Different-Embeddings.ipynb) produced errors when running the model and needed to be debugged. This required some time and effort to fix for all 3 embeddings. (Note: the final cell of the script also had a reference error, but this was an easy fix).
- The FastText model was missing from the GitHub link provided in the author's GitHub ReadMe page. After getting no response from the model creators, I noticed that someone else had raised this as an issue on the papers author's GitHub page and they provided a link to the embeddings.
- The volatility in the model results was not expected, rerunning the model produced varying results even after setting seed. After researching, it became apparent that performing hyperparameter tuning (as the authors did) may prove helpful. There were many parameters to tune which took time to run, and I was only able to get the results for Word2Vec and FastText within the limited time and colab runtime limits.
- If using Colab, it also takes some time to upload `all_hourly_data.csv` to Google Drive (I let it upload overnight).
- I also attempted to use transformers instead of convolution but the model was very slow, even on GPU and the runtime was disconnected before completing the 7th iteration of Word2Vec. The code is commented out and the output displayed in `all_models.ipynb` file for reference.


## Recommendations to original authors

Firstly, creating a GitHub page for this paper was very helpful and valuable. The paper was also generally well-written with a lot of detail. However, here are a few recommendations:

- To provide a requirements.txt file to ensure that the code can be run easily and ensure reproducibility.
- Specify all parameters in the paper including batch size, seeds, kernel size.



In [ ]:
# Checking that the whole notebook does not exceed 8 mins (not a hard rule but needs to run fast))
print("Total running time = {:.2f} minutes".format((time.time() - _START_RUNTIME)/60))

Total running time = 7.68 minutes


# References

[1] Johnson, A., Pollard, T., Shen, L. et al. MIMIC-III, a freely accessible critical care database. Sci Data 3, 160035 (2016). https://doi.org/10.1038/sdata.2016.35

[2] Shirly Wang, Matthew B. A. McDermott, Geeticka Chauhan, Michael C. Hughes, Tristan Naumann, and Marzyeh Ghassemi. MIMIC-Extract: A Data Extraction, Preprocessing, and Representation Pipeline for MIMIC-III. arXiv:1907.08322. https://arxiv.org/pdf/1907.08322.pdf

[3]  Kormilitzin A, Vaci N, Liu Q, Nevado-Holgado A. Med7: a transferable clinical natural language processing model for electronic health records. 2020 (arXiv preprint), arXiv:2003.01271. https://arxiv.org/pdf/2003.01271


